In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [ ]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [ ]:
link = 'https://fapergs.rs.gov.br/encerrados'

links_estados = []

parser_links = parser(link)

parser_links.xpath('/text()')

driver = webdriver.Firefox()
driver.get(link)

links_bruto = driver.find_elements_by_xpath('//article[@class="conteudo-lista__item clearfix"]//header//h2//a')

for link in links_bruto:
    links_estados.append(link.get_attribute('href'))

driver.quit()

links_estados.append('https://fapergs.rs.gov.br/em-julgamento')
links_estados.append('https://fapergs.rs.gov.br/abertos')

In [ ]:

#######################
#parser_lista_proyectos = parser()
fapergs = pd.DataFrame()

for link in links_estados:

    links_proyectos = []
    titulos_proyectos = []

    fapergs_estado = pd.DataFrame()

    driver = webdriver.Firefox()
    driver.get(link)

    bases = driver.find_elements_by_xpath('//article[@class="conteudo-lista__item clearfix"]//header//h2//a')

    for base in bases:
        links_proyectos.append(base.get_attribute('href'))
        titulos_proyectos.append(base.text)

    estado = driver.find_element_by_xpath('//article[@class="conteudo-lista__item clearfix"]//header//ul//li//a').text

    ano = driver.find_element_by_xpath('//article//header[@class="artigo__cabecalho"]//h1').text
    if ano == 'Em Julgamento' or ano == 'Abertos':
        ano = '2021'
    
    driver.quit()

    fapergs_estado['Título'] = titulos_proyectos
    fapergs_estado['Link'] = links_proyectos
    fapergs_estado['Año'] = ano
    fapergs_estado['Estado'] = estado

    fapergs = pd.concat([fapergs, fapergs_estado])

fapergs = fapergs.reset_index(drop=True) 

In [ ]:
pdfs_proyectos = []
descripcion_proyectos = []

for link in fapergs['Link']:
    driver = webdriver.Firefox()
    driver.get(link)
    ## PDFs
    pdfs = driver.find_elements_by_xpath('//div[@class="row"]//a')

    pdfs_proy = ''
    for p in pdfs:
        
        p = str(p.get_attribute('href'))
        
        if p.endswith('.pdf'):
            pdfs_proy = pdfs_proy + p + ', '

    pdfs_proyectos.append(pdfs_proy.strip(', '))

    #Descripción

    descripcion_bruta = driver.find_elements_by_xpath('//div[@class="row"]//p')
    descripcion = ''
    for d in descripcion_bruta:
        
        d = d.text
        descripcion = descripcion + d + ' '

    descripcion_proyectos.append(descripcion)

    driver.quit()
#print(pdfs_proyectos)
#print(descripcion_proyectos)
fapergs['Pdfs'] = pdfs_proyectos
fapergs['Descripción'] = descripcion_proyectos

In [ ]:
fapergs.to_excel('brasil_fapergs.xlsx')

In [29]:
fapergs[0:20]

,Título,Link,Año,Estado,Pdfs,Descripción
0,EDITAL FAPERGS 10/2020 - AUXÍLIO RECÉM-DOUTOR ...,https://fapergs.rs.gov.br/edital-fapergs-10-20...,2020,ENCERRADO,https://fapergs.rs.gov.br/upload/arquivos/2020...,
1,EDITAL 09/2020 - PROGRAMA TECHFUTURO,https://fapergs.rs.gov.br/edital-09-2020-progr...,2020,ENCERRADO,https://fapergs.rs.gov.br/upload/arquivos/2020...,EDITAL 09/2020 - PROGRAMA TECHFUTURO
2,CHAMADA Decit/SCTIE/MS-CNPq-FAPERGS 08/2020 – ...,https://fapergs.rs.gov.br/ppsus-2020,2020,ENCERRADO,https://fapergs.rs.gov.br/upload/arquivos/2020...,
3,EDITAL 07/2020 PROGRAMA FINEP – TECNOVA/RS 2ª ...,https://fapergs.rs.gov.br/edital-07-2020-progr...,2020,ENCERRADO,https://fapergs.rs.gov.br/upload/arquivos/2020...,Subvenção Econômica à Inovação Hotsite: Tecnov...
4,EDITAL EMERGENCIAL FAPERGS 06/2020 - CIÊNCIA E...,https://fapergs.rs.gov.br/edital-emergencial-f...,2020,ENCERRADO,https://fapergs.rs.gov.br/upload/arquivos/2020...,
5,EDITAL FAPERGS 05/2020 - PROGRAMA INSTITUCIONA...,https://fapergs.rs.gov.br/edital-fapergs-05-20...,2020,ENCERRADO,https://fapergs.rs.gov.br/upload/arquivos/2020...,
6,EDITAL FAPERGS 04/2020 - Apoio a Projetos de P...,https://fapergs.rs.gov.br/edital-fapergs-04-20...,2020,ENCERRADO,https://fapergs.rs.gov.br/upload/arquivos/2020...,
7,EDITAL 03/2020 - PROGRAMA INSTITUCIONAL DE BOL...,https://fapergs.rs.gov.br/edital-03-2020-progr...,2020,ENCERRADO,https://fapergs.rs.gov.br/upload/arquivos/2020...,
8,EDITAL 02/2020 - AUXÍLIO PARA ORGANIZAÇÃO DE E...,https://fapergs.rs.gov.br/edital-02-2020-auxil...,2020,ENCERRADO,https://fapergs.rs.gov.br/upload/arquivos/2019...,
9,EDITAL 01/2020 - AUXÍLIO À PARTICIPAÇÃO EM EVE...,https://fapergs.rs.gov.br/edital-01-2020-auxil...,2020,ENCERRADO,https://fapergs.rs.gov.br/upload/arquivos/2019...,
